In [1]:
import pm4py
import os
import numpy as np
import networkx as nx
import itertools as it
from operator import itemgetter
from refiningEventLabels.lib.graph.graphTool import graphTool 
from refiningEventLabels.lib.objects.customParameters import customParameters
from refiningEventLabels.lib.refinement.labelRefinement import verticalRefinement, horizontalRefinement
from refiningEventLabels.lib.eventLogProcessing.DBTool import DBTool 
from refiningEventLabels.lib.eventLogProcessing.postProcessing import eventLogRenaming
from refiningEventLabels.lib.costFunction.mappings import createEventIDs, positionsOfCandidates
from refiningEventLabels.lib.costFunction.cost import bestMappings
from pm4py.objects.log.importer.xes import factory as xes_import_factory

# Here we use the event log "running-example" which is also used for the documentation in pm4py. 

In [2]:
log = xes_import_factory.apply("refiningEventLabels/lib/running-example.xes")
orgLog = xes_import_factory.apply("refiningEventLabels/lib/running-example.xes")

# Preprocessing functions

In [3]:
# get the variants with the corresponding IDs (-> lookup-map)
# input: log (imported with xes_importer)
# stored as a dicctionary with variants as key and as value the list of case IDs that share the variant
from pm4py.algo.filtering.log.variants import variants_filter

def lookUpTable(log):
    variant=variants_filter.get_variants_from_log_trace_idx(log, parameters=None)
    return variant

# get the variants from the dicctionary created with lookUpTable() and omit the IDs
# input: dicctionary with variants as keys and case IDs as values
# stored as a list containing the variants as lists (-> list of lists)
def getVariants(dicctionary):
    variants=list(dicctionary.keys())
    return [variant.split(',') for variant in variants]

# Calculating mappings

In [4]:
def createEventIDs(variants=[]):
    seq = it.count()
    return [[(next(seq),event) for event in variant] for variant in variants]

def common_labels(variant1, variant2):
    var1 = [y[1] for (x, y) in enumerate(variant1)]
    var2 = [y[1] for (x, y) in enumerate(variant2)]
    return list(set(var1).intersection(var2))

#Args: variant1, variant2 as a list of tuples from createEventIDs(variants)
#Returns: number of unique events in variant1 and variant2
def getNumberOfCommonLabels(variant1=[], variant2=[]):
    s1 = set([b for (a,b) in variant1])
    s2 = set([b for (a,b) in variant2])
    
    return len(s1.intersection(s2))

def get_positions_label(string, variant):
    positions = []
    i = 0
    for x,y in enumerate(variant):
        if y[1] == string:
            positions.insert(i, y[0])
            i +=1
    return positions

#Args: variant1, variant2 as a list of tuples from createEventIDs(variants)
#Returns: list of lists with all possible mappings for variant1 and variant2
def possibleMappings(variant1=[], variant2=[]):
    matches = [(a,c) for (a,b) in variant1 for (c,d) in variant2 if b == d]
    n = getNumberOfCommonLabels(variant1, variant2)
    
    return [list(combi) for combi in it.combinations(matches, n)
                        if len(set(it.chain.from_iterable(combi))) == (2*n)]

def positions_of_candidates(candidates, variants):
    positions_of_candidates = []
    for variant in variants:
        labels = set(map(itemgetter(1), variant))
        for label in labels:
            if label in candidates:
                positions = get_positions_label(label, variant)
                positions_of_candidates.extend(positions)
    return positions_of_candidates

# Calculating costs

In [5]:
from itertools import combinations
def costStructure(variant1, variant2, mapping):
    cost_structure = 0
    combis = list(combinations(mapping, 2)) 
    for (pair1, pair2) in combis:
            distance1 = abs(pair1[0] - pair2[0])
            distance2 = abs(pair1[1] - pair2[1])
            cost_structure += abs(distance1 - distance2)/2
    return cost_structure

def context(variant):
    predecessors_list = []
    successors_list = []
    predecessors = []
    successors = []
    empty = []
    rest = list(map(itemgetter(1), variant[1:]))
    predecessors_list.insert(0,empty)
    successors_list.insert(0,rest)
    for index in range(1,len(variant)):
        pred_before = predecessors_list[index-1]
        succ_before = successors_list[index-1]
        last_label = [variant[index-1][1]]
        current_label = variant[index][1]
        #predecessors of current label are the predecessors of the last label plus last label
        predecessors_list.insert(index, pred_before + last_label)
        s_temp = succ_before.copy()
        s_temp.remove(current_label)
        #successors of current label are the successors of the last label minus current label
        successors_list.insert(index, s_temp) 
    for elem in predecessors_list:
        predecessors.append(set(elem))
    for elem2 in successors_list:
        successors.append(set(elem2))
        
    return predecessors, successors

def costNoMatch(variant1, variant2, mapping):
    mapped = set(common_labels(variant1, variant2)) #set of labels that were mapped
    unmapped1 = set(map(itemgetter(1), variant1)).difference(mapped) #set of unmapped labels in variant1
    unmapped2 = set(map(itemgetter(1), variant2)).difference(mapped) #set of unmapped labels in variant2
    firstId1 = variant1[0][0]
    firstId2 = variant2[0][0]
    pred1, succ1 = context(variant1)
    pred2, succ2 = context(variant2)
    np1 = 0
    ns1 = 0
    np2 = 0
    ns2 = 0
    for unmapped_label1 in unmapped1:
        positions1 = [x-firstId1 for x in get_positions_label(unmapped_label1, variant1)]
        for p1 in positions1:
            np1 += len(pred1[p1])
            ns1 += len(succ1[p1])
    for unmapped_label2 in unmapped2:
        positions2 = [x-firstId2 for x in get_positions_label(unmapped_label2, variant2)]
        for p2 in positions2:
            np1 += len(pred2[p2])
            ns1 += len(succ2[p2])
    sum = np1+np2+ns1+ns2
    return sum

def costMatched(variant1, variant2, mapping):
    firstId1 = variant1[0][0]
    firstId2 = variant2[0][0]
    pred1, succ1 = context(variant1)
    pred2, succ2 = context(variant2)
    sum = 0
    for pair in mapping:
        p1 = pair[0]-firstId1
        p2 = pair[1]-firstId2
        sum += len(pred1[p1])+len(pred2[p2])-2*len(pred1[p1].intersection(pred2[p2])) #number of distinct predecessors
        sum += len(succ1[p1])+len(succ2[p2])-2*len(succ1[p1].intersection(succ2[p2])) #number of distinct successors
    return sum

def costMapping(cp,variant1,variant2,mapping):
    
    wm = cp.getMatchWeight()
    ws = cp.getStructureWeight()
    wn = cp.getNoMatchWeight()
     
    cost_struc = costStructure(variant1, variant2, mapping)
    cost_nomatch = costNoMatch(variant1, variant2, mapping)
    cost_match = costMatched(variant1, variant2, mapping)
    return wm*cost_match + ws*cost_struc + wn*cost_nomatch

In [6]:
def optimalMapping(variants, variant1, variant2, matrixx, cp):
    pos_variant1 = variants.index(variant1)
    pos_variant2 = variants.index(variant2)
    possible_mappings = possibleMappings(variant1, variant2)
    if possible_mappings != []:
        best_mapping = possible_mappings[0]
        cost_best = costMapping(cp,variant1,variant2,best_mapping)
        for mapping in possible_mappings:
            cost_new = costMapping(cp,variant1,variant2,mapping)
            if cost_new < cost_best:
                best_mapping = mapping
                cost_best = cost_new
        matrixx[pos_variant1, pos_variant2] = cost_best #entry ij in matrix updated with best cost
        matrixx[pos_variant2, pos_variant1] = cost_best #entry ji in matrix updated with best cost
        #bestMappings.append((best_mapping,cost_best))
    else:
        matrixx[pos_variant1, pos_variant2] = -42 #entry ij in matrix updated with best cost
        matrixx[pos_variant2, pos_variant1] = -42 #entry ji in matrix updated with best cost
    return best_mapping, cost_best

# Testing functionalities with "running-example"

In [7]:
lookuptable = lookUpTable(log)
print("Look-Up Table:", "\n", lookuptable)

Look-Up Table: 
 {'register request,examine casually,check ticket,decide,reinitiate request,examine thoroughly,check ticket,decide,pay compensation': [0], 'register request,check ticket,examine casually,decide,pay compensation': [1], 'register request,examine thoroughly,check ticket,decide,reject request': [2], 'register request,examine casually,check ticket,decide,pay compensation': [3], 'register request,examine casually,check ticket,decide,reinitiate request,check ticket,examine casually,decide,reinitiate request,examine casually,check ticket,decide,reject request': [4], 'register request,check ticket,examine thoroughly,decide,reject request': [5]}


In [8]:
orig_variants = getVariants(lookuptable)
print("Original variants (without event IDs) : \n", orig_variants)

Original variants (without event IDs) : 
 [['register request', 'examine casually', 'check ticket', 'decide', 'reinitiate request', 'examine thoroughly', 'check ticket', 'decide', 'pay compensation'], ['register request', 'check ticket', 'examine casually', 'decide', 'pay compensation'], ['register request', 'examine thoroughly', 'check ticket', 'decide', 'reject request'], ['register request', 'examine casually', 'check ticket', 'decide', 'pay compensation'], ['register request', 'examine casually', 'check ticket', 'decide', 'reinitiate request', 'check ticket', 'examine casually', 'decide', 'reinitiate request', 'examine casually', 'check ticket', 'decide', 'reject request'], ['register request', 'check ticket', 'examine thoroughly', 'decide', 'reject request']]


In [9]:
variants = createEventIDs(orig_variants)
print("Variants with unique  event IDs: \n", variants)

Variants with unique  event IDs: 
 [[(0, 'register request'), (1, 'examine casually'), (2, 'check ticket'), (3, 'decide'), (4, 'reinitiate request'), (5, 'examine thoroughly'), (6, 'check ticket'), (7, 'decide'), (8, 'pay compensation')], [(9, 'register request'), (10, 'check ticket'), (11, 'examine casually'), (12, 'decide'), (13, 'pay compensation')], [(14, 'register request'), (15, 'examine thoroughly'), (16, 'check ticket'), (17, 'decide'), (18, 'reject request')], [(19, 'register request'), (20, 'examine casually'), (21, 'check ticket'), (22, 'decide'), (23, 'pay compensation')], [(24, 'register request'), (25, 'examine casually'), (26, 'check ticket'), (27, 'decide'), (28, 'reinitiate request'), (29, 'check ticket'), (30, 'examine casually'), (31, 'decide'), (32, 'reinitiate request'), (33, 'examine casually'), (34, 'check ticket'), (35, 'decide'), (36, 'reject request')], [(37, 'register request'), (38, 'check ticket'), (39, 'examine thoroughly'), (40, 'decide'), (41, 'reject re

In [10]:
cp = customParameters(candidateLabels = ["decide", "examine casually"],
                      horizontalThreshold = 0.5,
                      verticalThreshold = 0.3, 
                      weightStructure = 0.3, 
                      weightMatch = 0.3, 
                      weightNoMatch = 0.3)

count = len(variants) 
C = np.zeros((count,count)) 

In [11]:
var1 = variants[3]
var2 = variants[4]
print("Variant1: \n", var1)
print("Variant2: \n", var2)

mappings = possibleMappings(var1,var2)
print("\n Some mappings: \n", mappings [0], "\n", mappings[1], "\n")
print("Number of mappings: \n", len(mappings), "\n")

for mapping in possibleMappings(var1,var2):
    cost = costMapping(cp,var1,var2,mapping)
    print(cost)

#optimal = optimalMapping(var1, var2, C, wm, ws, wn, bestmappings)
#print("\n Best mapping: \n", optimal[0], "\n", "with cost:", optimal[1])

Variant1: 
 [(19, 'register request'), (20, 'examine casually'), (21, 'check ticket'), (22, 'decide'), (23, 'pay compensation')]
Variant2: 
 [(24, 'register request'), (25, 'examine casually'), (26, 'check ticket'), (27, 'decide'), (28, 'reinitiate request'), (29, 'check ticket'), (30, 'examine casually'), (31, 'decide'), (32, 'reinitiate request'), (33, 'examine casually'), (34, 'check ticket'), (35, 'decide'), (36, 'reject request')]

 Some mappings: 
 [(19, 24), (20, 25), (21, 26), (22, 27)] 
 [(19, 24), (20, 25), (21, 26), (22, 31)] 

Number of mappings: 
 27 

13.5
15.899999999999999
16.5
15.45
17.25
17.85
16.799999999999997
18.0
17.7
16.049999999999997
17.85
18.15
17.1
18.299999999999997
18.6
18.15
18.75
18.15
16.799999999999997
18.0
18.0
17.85
18.45
18.45
18.0
18.0
17.1


In [12]:
bestMappings = [] #list containing all best mappings

all_pairs = list(combinations(variants, 2))
for pair in all_pairs:
    optimal = optimalMapping(variants, pair[0],pair[1],C,cp)
    best_mapping = optimal[0]
    best_cost = optimal[1]
    bestMappings.append((best_mapping,best_cost))
    
maxCost = np.amax(C)
print("MaxCost used for normalization:", maxCost)
C = C/maxCost
print("Cost Matrix C: \n", C)
print("No of mappings:", len(bestMappings))

print(bestMappings)

MaxCost used for normalization: 22.35
Cost Matrix C: 
 [[0.         0.57718121 0.68456376 0.4966443  0.52348993 0.68456376]
 [0.57718121 0.         0.38926174 0.09395973 0.65771812 0.37583893]
 [0.68456376 0.38926174 0.         0.37583893 0.97986577 0.09395973]
 [0.4966443  0.09395973 0.37583893 0.         0.60402685 0.38926174]
 [0.52348993 0.65771812 0.97986577 0.60402685 0.         1.        ]
 [0.68456376 0.37583893 0.09395973 0.38926174 1.         0.        ]]
No of mappings: 15
[([(0, 9), (1, 11), (2, 10), (3, 12), (8, 13)], 12.899999999999999), ([(0, 14), (5, 15), (6, 16), (7, 17)], 15.299999999999999), ([(0, 19), (1, 20), (2, 21), (3, 22), (8, 23)], 11.1), ([(0, 24), (1, 25), (2, 26), (3, 27), (4, 28)], 11.7), ([(0, 37), (2, 38), (3, 40), (5, 39)], 15.299999999999999), ([(9, 14), (10, 16), (12, 17)], 8.7), ([(9, 19), (10, 21), (11, 20), (12, 22), (13, 23)], 2.0999999999999996), ([(9, 24), (10, 26), (11, 25), (12, 27)], 14.7), ([(9, 37), (10, 38), (12, 40)], 8.399999999999999), 

# Graph functions

In [13]:
candidates = cp.getCandidateLabels()

pos_candidates = positions_of_candidates(candidates, variants)
print("Positions of candidates: \n", pos_candidates)

Positions of candidates: 
 [1, 3, 7, 11, 12, 17, 20, 22, 25, 30, 33, 27, 31, 35, 40]


In [14]:
G = graphTool()
G.createGraphFromVariants(variants)
G.addOptimalMappings(bestMappings,maxCost,pos_candidates)

In [15]:
print(G.getGraph().nodes)#(data = True))
#print(G.getGraph().edges)
print(G.getGraph().edges(data=True))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
[(0, 1, {'weight': -1}), (0, 9, {'weight': 0.5771812080536912}), (0, 14, {'weight': 0.6845637583892616}), (0, 19, {'weight': 0.49664429530201337}), (0, 24, {'weight': 0.523489932885906}), (0, 37, {'weight': 0.6845637583892616}), (1, 2, {'weight': -1}), (1, 11, {'weight': 0.5771812080536912}), (1, 20, {'weight': 0.49664429530201337}), (1, 25, {'weight': 0.523489932885906}), (2, 3, {'weight': -1}), (2, 10, {'weight': 0.5771812080536912}), (2, 21, {'weight': 0.49664429530201337}), (2, 26, {'weight': 0.523489932885906}), (2, 38, {'weight': 0.6845637583892616}), (3, 4, {'weight': -1}), (3, 12, {'weight': 0.5771812080536912}), (3, 22, {'weight': 0.49664429530201337}), (3, 27, {'weight': 0.523489932885906}), (3, 40, {'weight': 0.6845637583892616}), (4, 5, {'weight': -1}), (4, 28, {'weight': 0.523489932885906}), (5, 6, {'weight': -1}), (

# Horizontal Refinement

In [16]:
def horizontalRefinement(cp, graphList):
    """
    Performs horizontal relabelling of event labels within a cluster; each event that belongs to the candidate labels will get a unique new label per cluster

    :param candidateLabels: s list of lsbels that should be refined
    :param graphList: a list of subgraphs where each subgraph represents a cluster of variants
    :return: s list of refined subgraphs, where the attribute 'newLabel' is changed for each candidate label, such that the event labels are unique per cluster
    """
    candidateLabels = cp.getCandidateLabels()
    
    counter=1
    for subgraph in graphList:
        for label in candidateLabels:
            for node, dict in list(subgraph.nodes(data=True)):
                if dict['curLabel'] == label:
                    dict['newLabel'] += str(counter)
        counter += 1

    return graphList

In [17]:
subgraphs = G.clusterDetection(cp)
print("First subgraph: \n", subgraphs[0].nodes(data=True))
print("\n Second subgraph: \n", subgraphs[1].nodes(data=True), "\n")

graphs=horizontalRefinement(cp, subgraphs)
print("\n refined first subgraph: \n", graphs[0].nodes(data=True))
print("\n refined second subgraph: \n",graphs[1].nodes(data=True))

print( "\n number of subgraphs:",len( subgraphs))

First subgraph: 
 [(0, {'curLabel': 'register request', 'newLabel': 'register request'}), (1, {'curLabel': 'examine casually', 'newLabel': 'examine casually'}), (2, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (3, {'curLabel': 'decide', 'newLabel': 'decide'}), (4, {'curLabel': 'reinitiate request', 'newLabel': 'reinitiate request'}), (5, {'curLabel': 'examine thoroughly', 'newLabel': 'examine thoroughly'}), (6, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (7, {'curLabel': 'decide', 'newLabel': 'decide'}), (8, {'curLabel': 'pay compensation', 'newLabel': 'pay compensation'}), (9, {'curLabel': 'register request', 'newLabel': 'register request'}), (10, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (11, {'curLabel': 'examine casually', 'newLabel': 'examine casually'}), (12, {'curLabel': 'decide', 'newLabel': 'decide'}), (13, {'curLabel': 'pay compensation', 'newLabel': 'pay compensation'}), (14, {'curLabel': 'register request', 'newLabel': 'register

# Vertical Refinement

In [18]:
db = DBTool(log)

In [19]:
#Get Connected components given a subgraph G
#Return a dictionary with the form {label: [{comp1},{comp2}...]}
def connectedComponents(G, candidateLabels):
    
    #Remove edges with 'weight' == -1
    G.remove_edges_from([(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] == -1])
    
    #1st find nodes with candidate labels, 2nd find connected components of each node, 3rd remove duplicate connected components 
    #since two connected nodes may have equal connected components   
    return {label : [list(cc) for cc in set([tuple(nx.node_connected_component(G, cnode[0]))
                                 for cnode in filter(lambda node: node[1]['curLabel'] == label, G.nodes(data=True))])]
                                     for label in candidateLabels}


#Get the size of the largest component given a connectedComponents dictionary
#Return a dictionary with the form {label: maxSize[{comp1},{comp2},...]}
def sizelargestComponent(connectedComponents):  
    return {label: len(max(cc, key=len, default=[])) 
                for label, cc in connectedComponents.items()}

#Get the average position of the events for a given connectedComponent, i.e., #Gi
#Return a list with the avg position [[avgPosComp1],[avgPosComp2],...]
def averagePosition(Gi, db):    
    return [sum(map(lambda eID: getPosition(eID,db), nodes))/len(nodes) 
                for nodes in Gi]

#Get the position of an event given its eventID
def getPosition(eID, db):   
    event = db.getEventByID(eID)
    return event.Position
    
#Sort the Connected components in ascending order
#Return a dictionary with sorted components having the form {label: [{comp1},{comp2}...]}
def sortConectedComponents(connectedComponents, db):   
    #sortCC = {event: sorted(zip(cc,averagePosition(cc,db)), key = lambda d: d[1]) 
     #           for event, cc in connectedComponents.items()}
        
    sortCC = {event: list(map(lambda d: d[0], sorted(zip(cc,averagePosition(cc,db)), key = lambda d: d[1])))
                 for event, cc in connectedComponents.items()}
    
    return  sortCC


#For each subgraph relabel candidateLabels according to the paper
def verticalRefinement(graphList, cp, db):
    
    candidateLabels = cp.getCandidateLabels()
    threshold = cp.getVerticalThreshold()
    
    for subgraph in graphList:
        cc = connectedComponents(subgraph, candidateLabels)
        cc = sortConectedComponents(cc, db)
        mSize = sizelargestComponent(cc)
        
        for event, nG in cc.items():
            for i,G in enumerate(nG, start = 1):
                for cn in G:
                    if i == 1 or len(G) >= threshold * mSize[event]:
                        subgraph.node[cn]['newLabel'] += '.' + str(i)
                        prevLabel = subgraph.node[cn]['newLabel']
                    else:
                        subgraph.node[cn]['newLabel'] = prevLabel
            prevLabel = '' 
                        
    return graphList
        

for g in verticalRefinement(subgraphs, cp, db):
    print(g.nodes(data=True), "\n")

[(0, {'curLabel': 'register request', 'newLabel': 'register request'}), (1, {'curLabel': 'examine casually', 'newLabel': 'examine casually1.1'}), (2, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (3, {'curLabel': 'decide', 'newLabel': 'decide1.1'}), (4, {'curLabel': 'reinitiate request', 'newLabel': 'reinitiate request'}), (5, {'curLabel': 'examine thoroughly', 'newLabel': 'examine thoroughly'}), (6, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (7, {'curLabel': 'decide', 'newLabel': 'decide1.3'}), (8, {'curLabel': 'pay compensation', 'newLabel': 'pay compensation'}), (9, {'curLabel': 'register request', 'newLabel': 'register request'}), (10, {'curLabel': 'check ticket', 'newLabel': 'check ticket'}), (11, {'curLabel': 'examine casually', 'newLabel': 'examine casually1.1'}), (12, {'curLabel': 'decide', 'newLabel': 'decide1.1'}), (13, {'curLabel': 'pay compensation', 'newLabel': 'pay compensation'}), (14, {'curLabel': 'register request', 'newLabel': 'register re

# PostProcessing

In [20]:
def postProcessingLog(subgraphList, db, eventLog, cp):
    labels = cp.getCandidateLabels()
    
    for subgraph in subgraphList:
        for eID, data in filter(lambda node: node[1]['curLabel'] in labels, subgraph.nodes(data=True)):            
            vID = db.getEventByID(eID).VariantID
            pos = db.getEventByID(eID).Position
            traces = db.getTracesByVariantID(vID)
            for t in traces:
                eventLog[t][pos]['concept:name'] = data['newLabel']

In [21]:
postProcessingLog(subgraphs, db, log, cp)

In [22]:
for case_index, case in enumerate(orgLog):
    print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
    for event_index, event in enumerate(case):
        print("event index: %d  event activity: %s" % (event_index, event["concept:name"]))


 case index: 0  case id: 3
event index: 0  event activity: register request
event index: 1  event activity: examine casually
event index: 2  event activity: check ticket
event index: 3  event activity: decide
event index: 4  event activity: reinitiate request
event index: 5  event activity: examine thoroughly
event index: 6  event activity: check ticket
event index: 7  event activity: decide
event index: 8  event activity: pay compensation

 case index: 1  case id: 2
event index: 0  event activity: register request
event index: 1  event activity: check ticket
event index: 2  event activity: examine casually
event index: 3  event activity: decide
event index: 4  event activity: pay compensation

 case index: 2  case id: 1
event index: 0  event activity: register request
event index: 1  event activity: examine thoroughly
event index: 2  event activity: check ticket
event index: 3  event activity: decide
event index: 4  event activity: reject request

 case index: 3  case id: 6
event ind

In [ ]:
for case_index, case in enumerate(log):
    print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
    for event_index, event in enumerate(case):
        print("event index: %d  event activity: %s" % (event_index, event["concept:name"]))